# Table of Contents
* [Learning Objectives](#Learning-Objectives)
* [Pandas for Reading and Writing Data](#Pandas-for-Reading-and-Writing-Data)
	* [Set-Up](#Set-Up)
	* [Example Data](#Example-Data)
* [Demonstration](#Demonstration)
	* [Creating a Temporary Directory](#Creating-a-Temporary-Directory)
	* [Reading and Inspecting](#Reading-and-Inspecting)
	* [Minimal Cleanup](#Minimal-Cleanup)
		* [Listing Columns](#Listing-Columns)
		* [Extracting Columns](#Extracting-Columns)
		* [Adding Columns](#Adding-Columns)
		* [Deleting Columns](#Deleting-Columns)
		* [Renaming Columns](#Renaming-Columns)
* [File Formats](#File-Formats)
	* [CSV](#CSV)
	* [CSV Online](#CSV-Online)
	* [Excel](#Excel)
	* [SQLite](#SQLite)
	* [JSON](#JSON)
	* [HDF](#HDF)
	* [FAQ: File Formats](#FAQ:-File-Formats)
* [Reading Large Data](#Reading-Large-Data)
	* [Read Performance](#Read-Performance)
		* [Timing Reads: Common Formats](#Timing-Reads:-Common-Formats)
		* [Timing Reads: Pickle and MSGPack](#Timing-Reads:-Pickle-and-MSGPack)
	* [Reading Compressed Data](#Reading-Compressed-Data)
		* [Set-Up](#Set-Up)
		* [Read Compressed file from Disk](#Read-Compressed-file-from-Disk)
		* [Read from URL and Extract to Disk](#Read-from-URL-and-Extract-to-Disk)
		* [Read from URL and Extract Without Disk](#Read-from-URL-and-Extract-Without-Disk)
	* [Reading Chunks](#Reading-Chunks)
* [Cleaning Temporary Files](#Cleaning-Temporary-Files)
* [Section Review](#Section-Review)


# Learning Objectives

After completing this notebook, the student will be able to:
* Use pandas to read and inspect file data
* Use pandas to read and write CSV, Excel, SQLite, and HDF files.
* Time the read and write operations so as to profile file I/O.
* Read larger data files, either as compressed files, or in chunks
* Select a file format more suited for a particular task.

# Pandas for Reading and Writing Data

pandas has excellent support for file I/O (read/write) for a wide variety of common file formats.

## Set-Up

In [ ]:
# Import modules needed for this lesson
import numpy as np
import pandas as pd
%matplotlib inline

# Set Jupyter notebook display options
pd.options.display.max_rows = 6
pd.options.display.max_columns = 8

# Test the version of pandas: it should be 0.16 or greater
pd.__version__

## Example Data

The data used throughout this demonstration is from the catalog of exoplanets -- planets outside our solar system -- compiled by [exoplanets.org](http://exoplanets.org/table)

# Demonstration

## Creating a Temporary Directory

Any time you are handling data, it is commonlly needed to handle "tmp" files.

In [ ]:
# Remove old tmp dir, Create a new tmp dir
import os
if not os.path.exists("tmp"):
    os.mkdir("tmp")

# Define a convenience function to help us clean up
def clean_tmp(file_name="tmp/exoplanet.csv"):
    if os.path.isfile(file_name):
        os.remove(file_name)
    elif os.path.exists(file_name):
        # if it is a directory, then do it recusively (you may have a __pycache__ folder there)
        for f in os.listdir(file_name):
            clean_tmp(os.path.join(file_name, f))
        os.rmdir(file_name)

## Reading and Inspecting

Below is a simple demonstration of reading and inspecting data.

We will see a much more detailed treatment of examining your data in the next lesson.

In [ ]:
file_name = "data/exoplanets.csv"
data = pd.read_csv(file_name)
print(type(data))

In [ ]:
# Preview data after a read.
# Notice the first column
data.iloc[0:2]

In [ ]:
# There is an Index array imbedded in the DataFrame
data.index

In [ ]:
# We can use a column to create a better Index
data = pd.read_csv(file_name, parse_dates=True, index_col='DATE')
data.head(10)     # another way to preview data after a read

In [ ]:
# Notice it is not just an Index, but a DatetimeIndex; note the dtype
data.index

In [ ]:
# You can sort by the Index
data = data.set_index(data.index.sort_values(ascending=False))
data.iloc[0:3]

## Minimal Cleanup

Later, we will cover data clean-up in much more detail. 

Here are just a few simple things to do to inspect and clean-up data.

### Listing Columns

In [ ]:
# View the labels of some or all of the columns
# data.columns[0:10]
data.columns

### Extracting Columns

In [ ]:
extract_data = data['RADIUS(rjupiter)']
print(type(extract_data))
extract_data.iloc[0:10]

### Adding Columns

- zero fill: 
    ```df['var'] = 0```
   - values from NumPy array: ```df['my_data'] = data```
   - note: df.var construct can not create a column by that
     name; only used to access existing columns by name

In [ ]:
# Add a column labeled 'price'
data['price'] = 1e6 # One *MILLION* dollars!

In [ ]:
# Inspect the updated DataFrame
data.price.iloc[0:5]

### Deleting Columns

In [ ]:
# Demonstration that you can delete columns after a file read
del data['FIRSTURL']

In [ ]:
# Inspect the results
data.iloc[0:5]

### Renaming Columns

In [ ]:
# Change the label of a single column
data = data.rename(columns={'NAME':'PLANET'})
data.iloc[0:3]

# File Formats

Pandas has an extensive set of I/O methods:
* Can read from a wide range of flat files, including Excel and HDF5
* Can also read SQL queries into memory
* Sensible defaults, automatically labeled and typed `DataFrames`

In [ ]:
# Print a list of all read_* methods in pandas

print("".join(["pd.%s\n" % reader 
               for reader in dir(pd) 
               if reader.startswith('read_')]))

In [ ]:
# DataFrame can be cast to files or other objects

print("".join(["pd.DataFrame.%s\n" % reader 
               for reader in dir(pd.DataFrame) 
               if reader.startswith('to_')]))

Below are external links to details on the file formats that pandas can read.

- [CSV](https://en.wikipedia.org/wiki/Comma-separated_values)
- Excel
- SQL
- [JSON](http://www.json.org/)
- [HDF5](https://www.hdfgroup.org/HDF5/)
- [pickle](https://docs.python.org/3/library/pickle.html)
- [msgpack](http://msgpack.org/)
- [Stata](https://en.wikipedia.org/wiki/Stata)
- [Google BigQuery](https://en.wikipedia.org/wiki/BigQuery)

We will now demonstrate using pandas reading and writing to a variety of file formats.

## CSV

http://pandas.pydata.org/pandas-docs/stable/io.html#csv-text-files

In [ ]:
df = pd.read_csv('data/exoplanets.csv', 
                 parse_dates=['DATE'],
                 encoding='utf-8')

In [ ]:
df.info()

In [ ]:
# The first exoplanet to get news headlines was in 1996.
# Use some indexing to inspect the rows.
# Combining head() and .iloc[] bracket indexing.
df.head(8).iloc[-1:]

Let's do some simple data extractions to prove we can.

In [ ]:
# Extract the name using [row index] and [column label]
df.loc[7,'NAME']

In [ ]:
# Extract the mass using the [column label] and [row index]
df.loc[7,'MASS(mjupiter)']

In [ ]:
# Extract the mass using and [row index] and [column index]
# The MASS(mjupiter) column is the 5th labeled column (not counting the Index)
# All pandas DataFrames have the method .iloc() 
df.iloc[7,5]

In [ ]:
df.to_csv('tmp/exoplanets.csv', index=False, encoding='utf-8')

## CSV Online

Pandas can read a data table directly from the web

In [ ]:
url = 'http://waterdatafortexas.org/reservoirs/individual/austin.csv'
df  = pd.read_csv(url, comment='#', index_col='date', parse_dates=True)
df

# note: you can parse dates, and ignore comment lines

## Excel

http://pandas.pydata.org/pandas-docs/stable/io.html#excel-files

In [ ]:
# Save the DataFrame as an Excel file
file_name = 'tmp/exoplanets.xls'
clean_tmp(file_name)

df.to_excel(file_name, index=False, encoding='utf-8')

In [ ]:
# Read the Excel file
file_name = 'tmp/exoplanets.xls'
df = pd.read_excel(file_name, sheetnames=[0], encoding='utf-8')
df.head(3)

## SQLite

http://pandas.pydata.org/pandas-docs/stable/io.html#sql-queries

In [ ]:
file_name = "tmp/exoplanets.sqlite"
clean_tmp(file_name)

from sqlalchemy import create_engine
engine = create_engine('sqlite:///' + file_name)

In [ ]:
df.to_sql('table', engine, index=False)

In [ ]:
df = pd.read_sql('table', engine)
df.head()

## JSON

http://pandas.pydata.org/pandas-docs/stable/io.html#json

In [ ]:
# Writing to JSON file
file_name = "tmp/exoplanets.json"
clean_tmp(file_name)

df.to_json(file_name)

In [ ]:
# JSON is ASCII so we can inspect the file with the shell
!head tmp/exoplanets.json

In [ ]:
# Reading the JSON file results in columns sorted by label

file_name = "tmp/exoplanets.json"
df = pd.read_json(file_name)
df.head()

In [ ]:
pd.read_json?

## HDF

http://pandas.pydata.org/pandas-docs/stable/io.html#hdf5-pytables

Pandas support 2 formats, ``fixed`` and ``table``. 

- ``fixed`` stores a heterogenous structure, but it can not be appended, nor queried, except via positional indexing (its much like a numpy record array).

- ``table`` stores a heterogenous structure, very much like a table. It can be appending row-wise, and queried similarly. 

In [ ]:
file_name = "tmp/exoplanets.hdf"
clean_tmp(file_name)

df.to_hdf(file_name,
          'df',
           mode='w',
           format='table',
           encoding='utf-8')

In [ ]:
file_name = "tmp/exoplanets.hdf"
df = pd.read_hdf(file_name,'df',encoding='utf-8')
df.head()

## FAQ: File Formats

Frequently asked file-format questions, and answers:

- which formats provide good fidelity
  - hdf5, pickle, msgpack
  
- which formats can you query
  - hdf5, sql
  
- which formats can you iterate
  - csv, hdf5, sql
  
- which formats provide better interoprability
  - csv, json, excel
  
- which formats can you transmit over the wire
  - json, msgpack
  
- which formats have better compression
  - hdf5, pickle, msgpack
  
- which formats allow multiple datasets in the same file
  - hdf5, msgpack

# Reading Large Data

Let's set some expectations for read times for various file formats

## Read Performance

### Timing Reads: Common Formats

In [ ]:
%timeit pd.read_excel('tmp/exoplanets.xls', sheetnames=[0])

In [ ]:
%timeit pd.read_sql('table', engine)

In [ ]:
%timeit pd.read_json('tmp/exoplanets.json')

In [ ]:
%timeit pd.read_csv('tmp/exoplanets.csv', parse_dates=['DATE'])

In [ ]:
%timeit pd.read_hdf('tmp/exoplanets.hdf','df')

### Timing Reads: Pickle and MSGPack

In [ ]:
df.to_pickle('tmp/exoplanets.pkl')
df.to_msgpack('tmp/exoplanets.msgpack',encoding='utf-8')

In [ ]:
%timeit pd.read_pickle('tmp/exoplanets.pkl')

In [ ]:
%timeit pd.read_msgpack('tmp/exoplanets.msgpack', encoding='utf-8')

## Reading Compressed Data

You may encounter compressed data files, e.g. ZIP or TAR.

Here we deomstrate some simple strategies for handling compressed data.

### Set-Up

In [ ]:
import requests, zipfile, io 

# Note: in python3 use io.BytesIO, in python2 use BytesIO.BytesIO

### Read Compressed file from Disk

Note that the ``infer_datetime_format=True`` interprets the datetime format as an ISO format.

In [ ]:
%%timeit -n 1 -r 1
# Just read from disk
chunks = pd.read_csv('data/201509-citibike-tripdata.csv.gz', 
                      index_col=0,
                      parse_dates=['starttime'],
                      infer_datetime_format=True)

### Read from URL and Extract to Disk

In [ ]:
%%timeit -n 1 -r 1
## Timing: about 15-180 seconds, depends on network, laptop 

# Get data, save to disk, read from disk
u = 'https://s3.amazonaws.com/tripdata/201509-citibike-tripdata.zip'
r = requests.get(u)
z = zipfile.ZipFile(io.BytesIO(r.content))

# Write to local disk, then read from local disk
z.extractall(path='tmp')
f = 'tmp/201509-citibike-tripdata.csv'
df = pd.read_csv(f)

### Read from URL and Extract Without Disk

In [ ]:
%%timeit -n 1 -r 1
## Timing: about 15-60 seconds, depending on network, laptop 

u  = 'https://s3.amazonaws.com/tripdata/201509-citibike-tripdata.zip'
r  = requests.get(u)
z  = zipfile.ZipFile(io.BytesIO(r.content))
f  = '201509-citibike-tripdata.csv' # name of file *INSIDE* the ZIP archive
df = pd.read_csv(z.open(f))

## Reading Chunks

In [ ]:
chunks = pd.read_csv('data/201509-citibike-tripdata.csv.gz', 
                      index_col=0,
                      parse_dates=['starttime'],
                      infer_datetime_format=True,
                      chunksize=10000)

for i, chunk in enumerate(chunks):
    print("%d -> %d" % (i, len(chunk)))
    # Note: for chunksize=10000, number of chunks is about 128


# Cleaning Temporary Files

Example: clean up tmp files created in this notebook

In [ ]:
# Always check for existence before removing
if os.path.exists("tmp"):
    os.listdir("tmp")

In [ ]:
# Build a list of files inside, remove each one by one, then remove the dir
if os.path.exists("tmp"):
    file_list = os.listdir("tmp")
    for file in file_list:
        clean_tmp(os.path.join("tmp", file))
    os.rmdir("tmp")
    
    
# other examples depend on ./tmp, so we recreate it here
os.mkdir('tmp')

# Section Review

Pandas Demomstration: Reading and Cleaning
* reading a CSV file
* column labels
* adding and removing columns

Pandas for Reading File Formats
* Excel
* HDF5
* MySQL
* JSON

Pandas for Reading Large Data
* timing read times for various formats
* reading from compressed files
* reading in chunks

***